In [1]:
import pandas as pd
import numpy as np
from scipy import io

In [2]:
posters = io.loadmat("../data/posters.mat")

In [3]:
posters["523162"].shape

(92, 140, 3)

In [57]:
movies = pd.read_csv("../data/tmdb_api_data.csv", lineterminator="\n")
movies.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/bOGkgRGdhrBYJSLpXaxhXVstddV.jpg,"[12, 28, 14]",299536,en,Avengers: Infinity War,As the Avengers and their allies have continue...,153.811,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,2018-04-25,Avengers: Infinity War,False,8.3,12490
1,False,/5zfVNTrkhMu673zma6qhFzG01ig.jpg,[878],300668,en,Annihilation,"A biologist signs up for a dangerous, secret e...",29.516,/d3qcpfNwbAMCNqWDHzPQsUYiUgS.jpg,2018-02-22,Annihilation,False,6.3,4232
2,False,/zjG95oDnBcFKMPgBEmmuNVOMC90.jpg,"[35, 18]",299782,en,The Other Side of the Wind,"Surrounded by fans and skeptics, grizzled dire...",6.820,/kFky1paYEfHxfCYByEc9g7gn6Zk.jpg,2018-11-02,The Other Side of the Wind,False,7.1,55
3,False,/q9hnJ9SzwcF30seRtXEzLd5l1gw.jpg,"[18, 35, 14]",351044,en,Welcome to Marwen,When a devastating attack shatters Mark Hoganc...,61.973,/o45VIAUYDcVCGuzd43l8Sr5Dfti.jpg,2018-12-21,Welcome to Marwen,False,6.6,174
4,False,/AmO8I38bkHwKhgxPNrd6djBQyPU.jpg,"[53, 9648, 27, 14]",361292,en,Suspiria,A darkness swirls at the center of a world-ren...,41.461,/dzWTnkert9EoiPWldWJ15dnfAFl.jpg,2018-10-11,Suspiria,False,7.2,579


In [58]:
movies = movies.dropna(subset=["id", "poster_path", "vote_average"])
movies["rating"] = np.round(movies["vote_average"])
movies.shape

(7353, 15)

In [62]:
all_ids = [str(i) for i in movies["id"] if str(i) in posters and posters[str(i)].shape == (92, 140, 3)]

In [63]:
len(np.unique(all_ids)) - len(all_ids)

-2137

In [64]:
ids = np.unique(all_ids)

In [65]:
X = np.stack([posters[i] for i in ids])

movies_by_id = movies.set_index("id")
y = np.stack([int(movies_by_id.loc[int(i),"rating"].iloc[0]) if type(movies_by_id.loc[int(i),"rating"]) \
              == pd.Series else int(movies_by_id.loc[int(i),"rating"]) for i in ids])

In [66]:
X.shape

(5123, 92, 140, 3)

In [67]:
y.shape

(5123,)

In [68]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)

In [69]:
from sklearn.model_selection import train_test_split

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y)

In [71]:
# Importing all necessary libraries 
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras import backend as K 

In [75]:
# train_data_dir = 'v_data/train'
# validation_data_dir = 'v_data/test'
# nb_train_samples =400
# nb_validation_samples = 100
epochs = 10
batch_size = 16
img_size = (92, 140)
input_shape = img_size + (3,)

In [80]:
def get_model():
    model = Sequential() 
    model.add(Conv2D(32, (2, 2), input_shape=input_shape)) 
    model.add(Activation('relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 

    model.add(Conv2D(32, (2, 2))) 
    model.add(Activation('relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 

    model.add(Conv2D(64, (2, 2))) 
    model.add(Activation('relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 

    model.add(Flatten()) 
    model.add(Dense(64)) 
    model.add(Activation('relu')) 
    model.add(Dropout(0.5)) 
    model.add(Dense(dummy_y.shape[1])) 
    model.add(Activation('sigmoid'))
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model


In [82]:
estimator = KerasClassifier(build_fn=get_model, epochs=5, batch_size=5, verbose=0)

In [83]:
kfold = KFold(n_splits=10, shuffle=True)

In [84]:
%%time
results = cross_val_score(estimator, X_train, y_train, cv=kfold, n_jobs=-1)

CPU times: user 163 ms, sys: 227 ms, total: 389 ms
Wall time: 15min 58s


In [85]:
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 64.47% (3.01%)


In [86]:
estimator.predict(X_test)

AttributeError: 'KerasClassifier' object has no attribute 'model'

In [30]:
img_size = (92, 140)
new_shape = (X.shape[0], img_size[0] * img_size[1] * 3)
X = np.reshape(X, new_shape)
X.shape

(7260, 38640)

In [32]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [44]:
dummy_y.shape

(7260, 11)

In [45]:
def base_model():
    model = Sequential()
    model.add(Dense(8, input_dim=X.shape[1], activation="relu"))
    model.add(Dense(dummy_y.shape[1], activation="softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [9]:

from tensorflow import keras
from tensorflow.keras import layers

img_size = (140, 92)

total_pixels = img_size[0] *img_size[1] * 3
fc_size = 512

inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')
x = layers.Flatten(name = 'flattened_img')(inputs) #turn image to vector.

x = layers.Dense(fc_size, activation='relu', name='first_layer')(x)
outputs = layers.Dense(1, activation='sigmoid', name='class')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
np.sum([np.shape(x) != (92, 140, 3) for x in X_train])

0

In [11]:
customAdam = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="mean_squared_error",
              # List of metrics to monitor
              metrics=["binary_crossentropy","mean_squared_error"])
              
print('# Fit model on training data')

history = model.fit(np.stack(X_train), 
                    y_train, 
                    batch_size=32, 
                    shuffle = True, #important since we loaded cats first, dogs second.
                    epochs=3,
                    validation_data=(np.stack(X_test), y_test))
                    
#Train on 4096 samples, validate on 2048 samples
#loss: 0.5000 - binary_crossentropy: 8.0590 - mean_squared_error: 0.5000 - val_loss: 0.5000 - 
#val_binary_crossentropy: 8.0591 - val_mean_squared_error: 0.5000

Instructions for updating:
Use tf.cast instead.
# Fit model on training data
Train on 5445 samples, validate on 1815 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
5445/5445 [==============================] - 48s 9ms/sample - loss: 12.3475 - binary_crossentropy: -20.5083 - mean_squared_error: 12.3475 - val_loss: 12.9036 - val_binary_crossentropy: -22.3281 - val_mean_squared_error: 12.9036
Epoch 2/3
5445/5445 [==============================] - 52s 10ms/sample - loss: 12.3401 - binary_crossentropy: -20.6300 - mean_squared_error: 12.3401 - val_loss: 12.9036 - val_binary_crossentropy: -22.3281 - val_mean_squared_error: 12.9036
Epoch 3/3
5445/5445 [==============================] - 47s 9ms/sample - loss: 12.3401 - binary_crossentropy: -20.6300 - mean_squared_error: 12.3401 - val_loss: 12.9036 - val_binary_crossentropy: -22.3281 - val_mean_squared_error: 12.9036


In [17]:
np.sum(model.predict(np.stack(X_test)) == 1)

1815

In [ ]:
fc_layer_size = 128
# img_size = img_size

conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')
conv_layer = layers.Conv2D(24, kernel_size=3, activation='relu')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)
conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

conv_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)

customAdam = keras.optimizers.Adam(lr=1e-6)
conv_model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["binary_crossentropy","mean_squared_error"])
              
#Epoch 5/5 loss: 1.6900 val_loss: 2.0413 val_mean_squared_error: 0.3688